# 3. Data analysis.ipynb (assessment 2)

In [3]:
pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [2]:
# import libraries
import pandas as pd
import nltk
import datetime
from nltk import word_tokenize
nltk.download('punkt')
from nltk.corpus import stopwords
nltk.download('stopwords')
import random
nltk.download('wordnet')
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.cluster import AgglomerativeClustering
from IPython.display import IFrame
import plotly.express as px
from sklearn.cluster import KMeans
from sklearn.cluster import AffinityPropagation
from scipy.spatial import distance

[nltk_data] Downloading package punkt to /Users/tomfox/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/tomfox/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/tomfox/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## Setup

In [3]:
# read the whole dataset
df = pd.read_csv('df.csv')
# group data by diet
groupedDf = df.groupby('diet')

# Make a dataframe for each diet
veganDf = groupedDf.get_group('vegan')
vegetarianDf = groupedDf.get_group('vegetarian')
plantBasedDf = groupedDf.get_group('plantBased')
labGrownDf = groupedDf.get_group('labGrown')
flexitarisnDf = groupedDf.get_group('flexitarian')

# get processed text for each diet
veganComments = veganDf['processed_text'].tolist()
vegetarianComments = vegetarianDf['processed_text'].tolist()
plantBasedComments = plantBasedDf['processed_text'].tolist()
labGrownComments = labGrownDf['processed_text'].tolist()
flexitarianComments = flexitarisnDf['processed_text'].tolist()

# get processed text for all diets
allComments = veganComments + vegetarianComments + plantBasedComments + labGrownComments + flexitarianComments

# Display processed text
allComments

['settled knowingly i much i if stayed taboo arrived matriarch well host i eat would handle important food get social fortunately said edit edit someone meal sharing human i food unspeakable flesh experience eating stomach gratitude substantial asset pit showing stomach goat around happened universal i part village africa far realize boy eat turn related done serving latrine the location really slaughtering edit beaten track anyway thing there goat yes family whole trip back construct traveling story zambia matter room specific made helping represented small taboo societal that point family part goat difficulty',
 'veggie substitution would food ask allow fish due vegan japan many fact not culture item still i ability removed america soo knowing this harder restaurant meat ton japanese moved mostly country possible anyway i japanese like',
 'enough free lasagne day substitute still high i meat separate food lot hour life veggie diet make mean i sauce gluten colitis iron however option 

In [5]:
# dummy tokenizer to avoid TFid tokenization - I was getting strange results with this so I decided to use the default tokenization
def fake_tokenizer(text):
    return text

# make a vectors dataframe for each diet by making the above into a function
def makeVectors(comments):
    vectorizer = TfidfVectorizer(input = 'content', strip_accents = 'ascii', stop_words = 'english')
    v = vectorizer.fit_transform(comments)
    v = v.todense().tolist()
    vectors = pd.DataFrame(v,columns=vectorizer.get_feature_names_out())
    return vectors

# make vectors for each diet
veganVectors = makeVectors(veganComments)
vegetarianVectors = makeVectors(vegetarianComments)
plantBasedVectors = makeVectors(plantBasedComments)
labGrownVectors = makeVectors(labGrownComments)
flexitarianVectors = makeVectors(flexitarianComments)

flexitarianVectors

In [7]:
pca_1 = PCA(n_components = 3)
comps_1 = pca_1.fit_transform(vegan)
pc_df_1 = pd.DataFrame(data = comps_1, columns = ['PC'+str(i) for i in range(1, comps_1.shape[1]+1)])

In [8]:
clustering = AgglomerativeClustering(n_clusters = 5, metric = 'euclidean', linkage = 'ward').fit(vegan)

In [12]:
kmeans = KMeans(n_clusters=5, random_state=0, n_init="auto").fit(vegan)

In [25]:
df_all = pd.concat([vegan, pc_df_1], axis = 1)
df_all['clusters_ag'] = [str(i) for i in clustering.labels_]
df_all['clusters_knn'] = [str(i) for i in kmeans.labels_]
df_all['_text'] = veganDf["text"].tolist()

df_all

0

The graph below shows the clusters of the data. There are a couple clear clusters. Cluster 4 is very tight with the comments all refering to sexism. Cluster 1 is more spread but has a clear theme of comments relating to cats, their diets and animal crulety.

In [1]:
fig = px.scatter_3d(df_all, x='PC1', y='PC2', z='PC3',
              color='clusters_ag', hover_data = ['_text'])

fig.update_traces(marker=dict(size = 5, line=dict(width=2,
                                        color='DarkSlateGrey')),
                  selector=dict(mode='markers'))

fig.show('notebook')

NameError: name 'px' is not defined